In [5]:
import os
import gzip
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from struct import unpack
from numpy import zeros, uint8, float32, ravel

In [8]:
#####################################################################################
# dataset used: http://yann.lecun.com/exdb/mnist/
# Prepare data for training and testing
#####################################################################################

datapath = '../datasets/mnist'
n_classes = 10

def get_labeled_data(imagefile, labelfile, picklename):
    imagefile = os.path.join(datapath, imagefile)
    labelfile = os.path.join(datapath, labelfile)
    picklename = os.path.join(datapath, picklename)
    
    if os.path.isfile('%s.pickle' % picklename):
        data = pickle.load(open('%s.pickle' % picklename, 'rb'))
    else:
        # Open the images with gzip in read binary mode
        images = gzip.open(imagefile, 'rb')
        labels = gzip.open(labelfile, 'rb')

        # Read the binary data

        # We have to get big endian unsigned int. So we need '>I'

        # Get metadata for images
        images.read(4)  # skip the magic_number
        number_of_images = images.read(4)
        number_of_images = unpack('>I', number_of_images)[0]
        rows = images.read(4)
        rows = unpack('>I', rows)[0]
        cols = images.read(4)
        cols = unpack('>I', cols)[0]

        # Get metadata for labels
        labels.read(4)  # skip the magic_number
        N = labels.read(4)
        N = unpack('>I', N)[0]

        if number_of_images != N:
            raise Exception('The number of labels did not match '
                            'the number of images.')

        # Get the data
        x = []
        y = []
        for i in range(N):
            if i % 1000 == 0:
                print("i: %i" % i)
            temp_x = []
            for row in range(rows):
                for col in range(cols):
                    tmp_pixel = images.read(1)  # Just a single byte
                    tmp_pixel = unpack('>B', tmp_pixel)[0]
                    temp_x.append(float(tmp_pixel) / 255)
            x.append(temp_x)
            tmp_label = labels.read(1)
            y.append(unpack('>B', tmp_label)[0])
        x = np.array(x)
        y = np.array(y)
        data = [x,y]
        pickle.dump(data, open("%s.pickle" % picklename, "wb"))
    return data

train_x, train_y = get_labeled_data('train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte.gz', 'train')
test_x, test_y = get_labeled_data('t10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz', 'test')

with tf.Session() as sess:
    train_y = sess.run(tf.one_hot(indices=train_y, depth=n_classes, on_value=1., off_value=0.))
    test_y = sess.run(tf.one_hot(indices=test_y, depth=n_classes, on_value=1., off_value=0.))


In [9]:
l_rate = 0.001
n_classes = 10
num_epochs = 10
batch_size = 128
nn_node_hl1 = 1500
nn_node_hl2 = 1500
nn_node_hl3 = 1500
num_features = len(train_x[0])

x = tf.placeholder('float', [None, num_features])
y = tf.placeholder('float')


def create_nueral_network(data):
    hidden_layer1 = {'weights': tf.Variable(tf.random_normal([num_features, nn_node_hl1])),
                    'biases': tf.Variable(tf.random_normal([nn_node_hl1]))}
    
    hidden_layer2 = {'weights': tf.Variable(tf.random_normal([nn_node_hl1, nn_node_hl2])),
                    'biases': tf.Variable(tf.random_normal([nn_node_hl2]))}
    
    hidden_layer3 = {'weights': tf.Variable(tf.random_normal([nn_node_hl2, nn_node_hl3])),
                    'biases': tf.Variable(tf.random_normal([nn_node_hl3]))}
    
    output_layer = {'weights': tf.Variable(tf.random_normal([nn_node_hl3, n_classes])),
                    'biases': tf.Variable(tf.random_normal([n_classes]))}
    
    l1 = tf.add(tf.matmul(data, hidden_layer1['weights']), hidden_layer1['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1, hidden_layer2['weights']), hidden_layer2['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2, hidden_layer3['weights']), hidden_layer3['biases'])
    l3 = tf.nn.relu(l3)
    
    output = tf.add(tf.matmul(l3, output_layer['weights']), output_layer['biases'])
    return output


def train_neural_network(x):
    prediction = create_nueral_network(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    optimize = tf.train.AdamOptimizer(learning_rate=l_rate).minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(num_epochs):
            epoch_loss = 0
            i = 0
            while i < len(train_x):
                start = i
                end = i + batch_size
                
                batch_x = train_x[start:end]
                batch_y = train_y[start:end]
                
                _, c = sess.run([optimize, cost], feed_dict={x: batch_x, y: batch_y})
                
                epoch_loss += c
                i += batch_size
            print('Epoch: ', epoch, ' Out of: ', num_epochs, ' loss: ', epoch_loss)
        
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        
        print('Accuracy: ', accuracy.eval({x: test_x, y: test_y}))
        

train_neural_network(x)

Epoch:  0  Out of:  10  loss:  5418978.34674
Epoch:  1  Out of:  10  loss:  1216854.67543
Epoch:  2  Out of:  10  loss:  560792.197189
Epoch:  3  Out of:  10  loss:  327129.762535
Epoch:  4  Out of:  10  loss:  231620.476891
Epoch:  5  Out of:  10  loss:  187364.331367
Epoch:  6  Out of:  10  loss:  159485.793633
Epoch:  7  Out of:  10  loss:  134365.779572
Epoch:  8  Out of:  10  loss:  110811.053268
Epoch:  9  Out of:  10  loss:  120428.96904
Accuracy:  0.9598
